In [1]:
import pickle
import os
import sys
import datetime
import pandas as pd
import numpy as np
import nltk.data
from gensim.models import Word2Vec
import multiprocessing
from copy import deepcopy
from sklearn.metrics import roc_auc_score    
from sklearn.cross_validation import StratifiedKFold

In [3]:
#on VM
os.chdir('/mnt/mypartition/Desktop2/pubmed_nlp_research/DeepLearningMovies_datasets/')
import KaggleWord2VecUtility

In [2]:
#on local:
sys.path.append('/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/')
from KaggleWord2VecUtility import KaggleWord2VecUtility
DATADIR='/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/'

In [2]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [3]:
#df = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/article_replication/df.p', 'rb'))
df = pickle.load(open('/mnt/mypartition/Desktop2/dfJournals_trans_categories.p', 'rb'))

In [5]:
#for the journals VM dataframe, change the label of the 'sentiment' column to 'label'
df.columns = ['label', 'title', 'abstract', 'qualifier_terms', 'descriptor_terms']

In [4]:
print df.shape
print df.columns.values

(97049, 5)
['sentiment' 'title' 'abstract' 'qualifier_terms' 'descriptor_terms']


In [6]:
df[0:10]

,label,title,abstract,qualifier_terms,descriptor_terms
0,T0,Quantification of the relative contributions o...,Amyotrophic lateral sclerosis (ALS) and fronto...,,
1,T0,Ubiquitin Specific Protease 36 (USP36) control...,Ubiquitination of the TrkA neurotrophin recept...,,
2,T0,Functional Diversity of Cytotoxic tRNase/immun...,Contact-dependent growth inhibition (CDI) is a...,,
3,T0,Identification of a distinct substrate binding...,The type III secretion system (T3SS) effector ...,,
4,T0,Role of chondroitin sulfate (CS) modification ...,Protein tyrosine phosphatase receptor type Z (...,,
5,T0,Chemically programmed bispecific antibodies in...,Chemically programmed bispecific antibodies (b...,,
6,T0,Non-mutagenic Suppression of Enterocyte Ferrop...,Iron transfer across the basolateral membrane ...,,
7,T0,Structural plasticity of cholesteryl ester tra...,Cholesteryl ester transfer protein (CETP) medi...,,
8,T0,Identification of a Membrane-Bound Prepore Spe...,Pore-forming toxins (PFT) are cytolytic protei...,,
9,T0,EspR-dependent ESAT-6 secretion of Mycobacteri...,Attenuation of M. bovis BCG strain is related ...,,


## Apply word2vec inversion

In [7]:
"""
docprob takes two lists
* docs: a list of documents, each of which is a list of sentences
* models: the candidate word2vec models (each potential class)

it returns the array of class probabilities.  Everything is done in-memory.
"""

def docprob(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = prob.groupby("doc").mean()
    return prob

In [14]:
a = datetime.datetime.now().replace(microsecond=0)

#5-fold stratified cross validation
from sklearn.cross_validation import StratifiedKFold

#add boolean labels
df['boolean_label'] = 1
for row in df.iterrows():
    if row[1]['label'] != 'T1/T2':
        df.set_value(row[0], 'boolean_label', 0)

#because no validation set, 4/5 of values go to train and 1/5 to test
#is this too high?***
#even though we shuffle, not as randomly distributed as the former method was
y = df['boolean_label'].values
skf = StratifiedKFold(y, n_folds=5, shuffle=True)

i = 1
avg = []
for train_index, test_index in skf:

    print("ROUND", i)
    i = i + 1
    #use the indexes to subset the df pandas dataframe
    train1, test1 = df.iloc[train_index], df.iloc[test_index]

    # ****** Split the training set into clean sentences
    #
    sentences_pos = []  # Initialize an empty list of sentences
    sentences_neg = []  # Initialize an empty list of sentences

    #here change to include all journal name labels of positive and negative
    inxs_pos = np.where(train1['boolean_label'] == 1)[0].tolist()
    inxs_neg = np.where(train1['boolean_label'] == 0)[0].tolist()

    print "Parsing sentences from training set"
    for inx in inxs_pos:
        review = train1["abstract"].iloc[inx]
        title = train1['title'].iloc[inx]
        #descriptors = train1['descriptor_terms'].iloc[inx]
        #qualifiers = train1['qualifier_terms'].iloc[inx]
        sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
        sentences_pos += KaggleWord2VecUtility.review_to_sentences(title, tokenizer)
        #sentences_pos += KaggleWord2VecUtility.review_to_sentences(descriptors, tokenizer)
        #sentences_pos += KaggleWord2VecUtility.review_to_sentences(qualifiers, tokenizer)


    for inx in inxs_neg:
        review = train1["abstract"].iloc[inx]
        title = train1['title'].iloc[inx]
        #descriptors = train1['descriptor_terms'].iloc[inx]
        #qualifiers = train1['qualifier_terms'].iloc[inx]
        sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
        sentences_neg += KaggleWord2VecUtility.review_to_sentences(title, tokenizer)
        #sentences_neg += KaggleWord2VecUtility.review_to_sentences(descriptors, tokenizer)
        #sentences_neg += KaggleWord2VecUtility.review_to_sentences(qualifiers, tokenizer)

    # ****** Split the labeled and unlabeled training sets into clean sentences
    #
    sentences = []  # Initialize an empty list of sentences

    for j in range(len(train1)):
        review = train1["abstract"].iloc[j]
        title = train1['title'].iloc[j]
        #descriptors = train1['descriptor_terms'].iloc[j]
        #qualifiers = train1['qualifier_terms'].iloc[j]

        sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
        sentences += KaggleWord2VecUtility.review_to_sentences(title, tokenizer)
        #sentences += KaggleWord2VecUtility.review_to_sentences(descriptors, tokenizer)
        #sentences += KaggleWord2VecUtility.review_to_sentences(qualifiers, tokenizer)

    print "Building and training w2v models"
    ## create a w2v learner 
    basemodel = Word2Vec(
        workers=multiprocessing.cpu_count(), # use your cores
        iter=3, # iter = sweeps of SGD through the data; more is better
        hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
        )
    basemodel.build_vocab(sentences) 

    #train models
    models = [deepcopy(basemodel) for y in range(2)]
    models[0].train(sentences_neg, total_examples=len(sentences_neg) )
    models[1].train(sentences_pos, total_examples=len(sentences_pos) )

    print "Parsing test sentences"
    # read in the test set as a list of a list of words

    docs = []
    for k in range(len(test1)):
        sents = []
        review = test1["abstract"].iloc[k]
        title = test1['title'].iloc[k]
        #descriptors = test1['descriptor_terms'].iloc[k]
        #qualifiers = test1['qualifier_terms'].iloc[k]

        sents += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
        sents += KaggleWord2VecUtility.review_to_sentences(title, tokenizer)
        #sents += KaggleWord2VecUtility.review_to_sentences(descriptors, tokenizer)
        #sents += KaggleWord2VecUtility.review_to_sentences(qualifiers, tokenizer)
        docs.append(sents)

    print "scoring test set"
    # get the probs (note we give docprob our test set plus the models)
    probs = docprob(docs,models).astype(object)

    predictions = np.ones((probs.shape[0]))

    predictions[np.where(probs.iloc[:,0] > 0.5)] = 0 # The first column is the negative model
    print predictions
    print np.size(predictions)

    score = roc_auc_score(test1["boolean_label"], predictions)
    #score = np.size(np.where(predictions == test1["sentiment"]))*1./np.size(predictions)
    print score
    #append to average
    avg.append(score)


print("average of 5 rotations:", sum(avg)/float(len(avg)))
    
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

In [10]:
def stratified_5_fold_cv(df, mesh_terms, title, label):
    #5-fold stratified cross validation

    #add boolean labels
    df['boolean_label'] = 1
    for row in df.iterrows():
        if row[1]['label'] != label:
            df.set_value(row[0], 'boolean_label', 0)

    #because no validation set, 4/5 of values go to train and 1/5 to test
    #is this too high?***
    #even though we shuffle, not as randomly distributed as the former method was
    y = df['boolean_label'].values
    skf = StratifiedKFold(y, n_folds=5, shuffle=True)

    i = 1
    avg = []
    for train_index, test_index in skf:
        print("ROUND", i)
        i = i + 1
        
        if i == 2:
            #use the indexes to subset the df pandas dataframe
            train1, test1 = df.iloc[train_index], df.iloc[test_index]

            # ****** Split the training set into clean sentences
            #
            sentences_pos = []  # Initialize an empty list of sentences
            sentences_neg = []  # Initialize an empty list of sentences

            #here change to include all journal name labels of positive and negative
            inxs_pos = np.where(train1['boolean_label'] == 1)[0].tolist()
            inxs_neg = np.where(train1['boolean_label'] == 0)[0].tolist()

            print "Parsing sentences from training set"
            for inx in inxs_pos:
                review = train1["abstract"].iloc[inx]
                sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
                if title:
                    title = train1['title'].iloc[inx]
                    sentences_pos += KaggleWord2VecUtility.review_to_sentences(title, tokenizer)
                if mesh_terms:
                    descriptors = train1['descriptor_terms'].iloc[inx]
                    qualifiers = train1['qualifier_terms'].iloc[inx]
                    sentences_pos += KaggleWord2VecUtility.review_to_sentences(descriptors, tokenizer)
                    sentences_pos += KaggleWord2VecUtility.review_to_sentences(qualifiers, tokenizer)        

            for inx in inxs_neg:
                review = train1["abstract"].iloc[inx]
                sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
                if title:
                    title = train1['title'].iloc[inx]
                    sentences_neg += KaggleWord2VecUtility.review_to_sentences(title, tokenizer)
                if mesh_terms:
                    descriptors = train1['descriptor_terms'].iloc[inx]
                    qualifiers = train1['qualifier_terms'].iloc[inx]
                    sentences_neg += KaggleWord2VecUtility.review_to_sentences(descriptors, tokenizer)
                    sentences_neg += KaggleWord2VecUtility.review_to_sentences(qualifiers, tokenizer)

            # ****** Split the labeled and unlabeled training sets into clean sentences
            #
            sentences = []  # Initialize an empty list of sentences

            for j in range(len(train1)):
                review = train1["abstract"].iloc[j]
                sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
                if title:
                    title = train1['title'].iloc[j]
                    sentences += KaggleWord2VecUtility.review_to_sentences(title, tokenizer)
                if mesh_terms:
                    descriptors = train1['descriptor_terms'].iloc[j]
                    qualifiers = train1['qualifier_terms'].iloc[j]
                    sentences += KaggleWord2VecUtility.review_to_sentences(descriptors, tokenizer)
                    sentences += KaggleWord2VecUtility.review_to_sentences(qualifiers, tokenizer)

            print "Building and training w2v models"
            ## create a w2v learner 
            basemodel = Word2Vec(
                workers=multiprocessing.cpu_count(), # use your cores
                iter=3, # iter = sweeps of SGD through the data; more is better
                hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
                )
            basemodel.build_vocab(sentences) 

            #train models
            models = [deepcopy(basemodel) for y in range(2)]
            models[0].train(sentences_neg, total_examples=len(sentences_neg) )
            models[1].train(sentences_pos, total_examples=len(sentences_pos) )

            print "Parsing test sentences"
            # read in the test set as a list of a list of words

            docs = []
            for k in range(len(test1)):
                sents = []
                review = test1["abstract"].iloc[k]
                sents += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
                if title:
                    title = test1['title'].iloc[k]
                    sents += KaggleWord2VecUtility.review_to_sentences(title, tokenizer)
                if mesh_terms:
                    descriptors = test1['descriptor_terms'].iloc[k]
                    qualifiers = test1['qualifier_terms'].iloc[k]
                    sents += KaggleWord2VecUtility.review_to_sentences(descriptors, tokenizer)
                    sents += KaggleWord2VecUtility.review_to_sentences(qualifiers, tokenizer)
                docs.append(sents)

            print "scoring test set"
            # get the probs (note we give docprob our test set plus the models)
            probs = docprob(docs,models).astype(object)

            predictions = np.ones((probs.shape[0]))

            predictions[np.where(probs.iloc[:,0] > 0.5)] = 0 # The first column is the negative model
            print predictions
            print np.size(predictions)

            score = roc_auc_score(test1["boolean_label"], predictions)
            #score = np.size(np.where(predictions == test1["sentiment"]))*1./np.size(predictions)
            print score
            #append to average
            #avg.append(score)


    #print("average of 5 rotations:", sum(avg)/float(len(avg)))
    
    return models

In [39]:
a = datetime.datetime.now().replace(microsecond=0)

res = stratified_5_fold_cv(df, False, True, 'T0')

b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

('ROUND', 1)
Parsing sentences from training set
Building and training w2v models
Parsing test sentences
scoring test set
[ 1.  1.  1. ...,  0.  0.  0.]
19411
0.942976797945
('ROUND', 2)
('ROUND', 3)
('ROUND', 4)
('ROUND', 5)
0:09:55


In [42]:
pickle.dump(res, open('/mnt/mypartition/Desktop2/pubmed_nlp_research/pickled_objects/w2vModelWithStopsT0.p', 'wb'))

In [15]:
a = datetime.datetime.now().replace(microsecond=0)

res1 = train_model(df, False, 'T1/T2')
pickle.dump(res1, open('/mnt/mypartition/Desktop2/pubmed_nlp_research/pickled_objects/w2vModelWithStopsT1T2.p', 'wb'))
    
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

Parsing sentences from training set


/home/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.trialregister.nl/trialreg/admin/rctview.asp?TC=1173." looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/home/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.controlled-trials.com/isrctn/pf/24165302." looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beauti

Building and training w2v models
0:09:00


In [16]:
a = datetime.datetime.now().replace(microsecond=0)

res2 = train_model(df, False, 'T3/T4')
pickle.dump(res2, open('/mnt/mypartition/Desktop2/pubmed_nlp_research/pickled_objects/w2vModelWithStopsT3T4.p', 'wb'))
    
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

Parsing sentences from training set
Building and training w2v models
0:08:57


In [14]:
#Define our function
def train_model(df, mesh_terms, label):

    #append 2-class labels
    df['boolean_label'] = 1
    for row in df.iterrows():
        if row[1]['label'] != label:
            df.set_value(row[0], 'boolean_label', 0)

    y = df['boolean_label'].values
    skf = StratifiedKFold(y, n_folds=5, shuffle=True)

    j = 1
    for train_index, test_index in skf:
        if j == 1:
            j = j + 1
            #use the indexes to subset the df pandas dataframe
            train1, test1 = df.iloc[train_index], df.iloc[test_index]

            # ****** Split the training set into clean sentences
            #
            sentences_pos = []  # Initialize an empty list of sentences
            sentences_neg = []  # Initialize an empty list of sentences

            #here change to include all journal name labels of positive and negative
            inxs_pos = np.where(train1['boolean_label'] == 1)[0].tolist()
            inxs_neg = np.where(train1['boolean_label'] == 0)[0].tolist()
            
            print "Parsing sentences from training set"
            for inx in inxs_pos:
                review = train1["abstract"].iloc[inx]
                title = train1['title'].iloc[inx]
                sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
                sentences_pos += KaggleWord2VecUtility.review_to_sentences(title, tokenizer)
                if mesh_terms:
                    descriptors = train1['descriptor_terms'].iloc[inx]
                    qualifiers = train1['qualifier_terms'].iloc[inx]
                    sentences_pos += KaggleWord2VecUtility.review_to_sentences(descriptors, tokenizer)
                    sentences_pos += KaggleWord2VecUtility.review_to_sentences(qualifiers, tokenizer)
                
            for inx in inxs_neg:
                review = train1["abstract"].iloc[inx]
                title = train1['title'].iloc[inx]
                sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
                sentences_neg += KaggleWord2VecUtility.review_to_sentences(title, tokenizer)
                if mesh_terms:
                    descriptors = train1['descriptor_terms'].iloc[inx]
                    qualifiers = train1['qualifier_terms'].iloc[inx]
                    sentences_neg += KaggleWord2VecUtility.review_to_sentences(descriptors, tokenizer)
                    sentences_neg += KaggleWord2VecUtility.review_to_sentences(qualifiers, tokenizer)
                
            # ****** Split the labeled and unlabeled training sets into clean sentences
            #
            sentences = []  # Initialize an empty list of sentences

            for j in range(len(train1)):
                review = train1["abstract"].iloc[j]
                title = train1['title'].iloc[j]
                sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
                sentences += KaggleWord2VecUtility.review_to_sentences(title, tokenizer)
                if mesh_terms:
                    descriptors = train1['descriptor_terms'].iloc[j]
                    qualifiers = train1['qualifier_terms'].iloc[j]
                    sentences += KaggleWord2VecUtility.review_to_sentences(descriptors, tokenizer)
                    sentences += KaggleWord2VecUtility.review_to_sentences(qualifiers, tokenizer)

            print "Building and training w2v models"
            ## create a w2v learner 
            basemodel = Word2Vec(
                workers=multiprocessing.cpu_count(), # use your cores
                iter=3, # iter = sweeps of SGD through the data; more is better
                hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
                )
            basemodel.build_vocab(sentences) 

            #train models
            models = [deepcopy(basemodel) for y in range(2)]
            models[0].train(sentences_neg, total_examples=len(sentences_neg) )
            models[1].train(sentences_pos, total_examples=len(sentences_pos) )
            
    return models

### External validation on paper's dataset

In [18]:
#df_val = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/article_replication/df.p', 'rb'))
df_val = pickle.load(open('/mnt/mypartition/Desktop2/pubmed_nlp_research/article_replication/df.p', 'rb'))

In [19]:
#check that column names are the same
print df_val.shape
print df_val.columns.values

(354, 6)
['pmid' 'title' 'abstract' 'qualifier_terms' 'descriptor_terms' 'label']


Get test set parsed and scored.

In [20]:
print "Parsing test sentences"
# read in the test set as a list of a list of words

docs = []
for k in range(len(df_val)):
    sents = []
    review = df_val["abstract"].iloc[k]
    title = df_val['title'].iloc[k]
    #descriptors = df_val['descriptor_terms'].iloc[k]
    #qualifiers = df_val['qualifier_terms'].iloc[k]
    sents += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
    sents += KaggleWord2VecUtility.review_to_sentences(title, tokenizer)
    #sents += KaggleWord2VecUtility.review_to_sentences(descriptors, tokenizer)
    #sents += KaggleWord2VecUtility.review_to_sentences(qualifiers, tokenizer)
    docs.append(sents)

Parsing test sentences


In [21]:
res = pickle.load(open('/mnt/mypartition/Desktop2/pubmed_nlp_research/pickled_objects/w2vModelWithStopsT0.p', 'rb'))

In [22]:
#Score validation set using models built for TO
#Add labels to df
df_val['boolean_label'] = 1
for row in df_val.iterrows():
    if row[1]['label'] != 'T0':
        df_val.set_value(row[0], 'boolean_label', 0)

In [23]:
print "scoring test set"
# get the probs (note we give docprob our test set plus the models)
probs = docprob(docs,res).astype(object)
print probs[0:10]

predictions = np.ones((probs.shape[0]))

predictions[np.where(probs.iloc[:,0] > 0.5)] = 0 # The first column is the negative model
print predictions
print np.size(predictions)

score = roc_auc_score(df_val["boolean_label"], predictions)
print score

scoring test set
              0            1
doc                         
0      0.649182     0.350818
1    0.00391977      0.99608
2      0.831252     0.168748
3       0.84759      0.15241
4      0.999662  0.000338351
5      0.300792     0.699208
6      0.999924  7.55319e-05
7      0.341294     0.658706
8             1  3.87099e-08
9       0.96874    0.0312598
[ 0.  1.  0.  0.  0.  1.  0.  1.  0.  0.  1.  1.  0.  1.  1.  0.  1.  1.
  0.  1.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  0.  1.  1.  0.  0.  1.
  0.  0.  1.  1.  0.  0.  1.  0.  0.  1.  1.  0.  0.  1.  1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  1.  1.  0.  1.  1.  0.
  0.  0.  1.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.
  1.  0.  0.  0.  1.  0.  1.  1.  0.  0.  0.  1.  0.  1.  0.  1.  1.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

In [24]:
#Score validation set using models built for T1/T2
#Add labels to df
df_val['boolean_label'] = 1
for row in df_val.iterrows():
    if row[1]['label'] != 'T1/T2':
        df_val.set_value(row[0], 'boolean_label', 0)

In [25]:
print "scoring test set"
# get the probs (note we give docprob our test set plus the models)
probs = docprob(docs,res1).astype(object)
print probs[0:10]

predictions = np.ones((probs.shape[0]))

predictions[np.where(probs.iloc[:,0] > 0.5)] = 0 # The first column is the negative model
print predictions
print np.size(predictions)

score = roc_auc_score(df_val["boolean_label"], predictions)
print score

scoring test set
            0           1
doc                      
0    0.697497    0.302503
1    0.961737   0.0382625
2    0.717966    0.282034
3    0.605429    0.394571
4    0.821622    0.178378
5    0.993986  0.00601392
6    0.837053    0.162947
7    0.744812    0.255188
8    0.994281  0.00571881
9    0.967571   0.0324289
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.
  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  1.
  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  1.  0.  1.  0.  0.  0.  0.  1.
  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  1.  1.  1.  1.  0.  1.  0.  1.  1.  1.  0.  1.  1.  1.
  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  1.  1.  0.  1.  1.  1.  0.
  0.  0.  0.  

In [26]:
#Score validation set using models built for T3/T4
#Add labels to df
df_val['boolean_label'] = 1
for row in df_val.iterrows():
    if row[1]['label'] != 'T3/T4':
        df_val.set_value(row[0], 'boolean_label', 0)

In [27]:
print "scoring test set"
# get the probs (note we give docprob our test set plus the models)
probs = docprob(docs,res2).astype(object)
print probs[0:10]

predictions = np.ones((probs.shape[0]))

predictions[np.where(probs.iloc[:,0] > 0.5)] = 0 # The first column is the negative model
print predictions
print np.size(predictions)

score = roc_auc_score(df_val["boolean_label"], predictions)
print score

scoring test set
              0         1
doc                      
0      0.515032  0.484968
1      0.841101  0.158899
2      0.654105  0.345895
3      0.740195  0.259805
4      0.304986  0.695014
5      0.668322  0.331678
6      0.137801  0.862199
7      0.756234  0.243766
8    0.00119575  0.998804
9      0.340777  0.659223
[ 0.  0.  0.  0.  1.  0.  1.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  1.  1.  0.  0.  0.  1.  0.  1.  1.  0.  0.  1.  1.  0.  0.  0.  1.  1.
  1.  0.  0.  0.  1.  1.  0.  1.  0.  0.  0.  1.  1.  0.  1.  0.  0.  0.
  1.  1.  1.  1.  1.  0.  1.  1.  0.  0.  0.  1.  0.  0.  1.  0.  0.  1.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  1.  1.  0.  0.  0.
  0.  1.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.  1.
  1.  0.  1.  1.  0.  1.  0.  0.  1.  1.  1.  0.  1.  1.  1.  0.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  0.  0.  1.  1.  0.  0.  1.  0.  0.  0.  0.
  0.  0.  1.  

## Performance with abstract, title, and mesh terms on PAPER'S DATASET (much worse than BOWs/RF because so few records)
T0 class- 0.68 (0.63, 0.73)

T1/T2 class- 0.49 (0.45, 0.52)

T3/T4 class- 0.51 (0.48, 0.55)

## Performance with abstract, title, and mesh terms on PUBMED JOURNALS (performs how compared to BOWs/RF?)
T0 class- 0.94

T1/T2 class- 0.82

T3/T4 class- 0.88

### Import KaggleWord2VecUtility since didn't work from file
### Note: remove_stopwords set to True

In [8]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords


class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)
    
    @staticmethod
    def review_to_words( review, remove_stopwords=False ):
        # Function to convert a raw review to a string of words
        # The input is a single string (a raw movie review), and 
        # the output is a single string (a preprocessed movie review)
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text() 
        #
        # 2. Remove non-letters        
        review_text = re.sub("[^a-zA-Z]", " ", review_text) 
        #
        # 3. Convert to lower case, split into individual words
        words = review_text.lower().split()                             
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]   
        #
        # 6. Join the words back into one string separated by space, 
        # and return the result.
        return( " ".join( words ))   

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences